<a href="https://colab.research.google.com/github/shaghayegh5ghasemi/CIFAR10_train_test/blob/main/CNN_CIFAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchsummary

**Import requirements**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchsummary import summary

**Load data and define hyperparameters**

In [ ]:
# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# load our dataset
train_data = datasets.CIFAR10(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_laoder = DataLoader(dataset=train_data, batch_size=5, shuffle=True)
test_data = datasets.CIFAR10(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(dataset=test_data, batch_size=1, shuffle=True)

# hyperparameters
learning_rate = 0.001
batch_size = 64
num_epochs = 20

**Implement CNN model**

In [15]:
# our CNN model with three Layers and Relu as an activation function
class CNN(nn.Module):
  def __init__(self, channels = 3, num_classes = 10):
    super().__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(in_channels = channels, out_channels = 16, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.layer2 = nn.Sequential(
        nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.layer3 = nn.Sequential(
        nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 1, padding = 1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.flat = nn.Flatten()
    self.linear = nn.Linear(64*4*4, num_classes)
    self.softmax = nn.Softmax(dim=1)

  def forward(self, x):
    x = self.layer1(x)
    x = self.layer2(x)
    x = self.layer3(x)
    x = self.flat(x)
    x = self.linear(x)
    x = self.softmax(x)
    return x

In [14]:
cnn_model = CNN().to(device)
summary(cnn_model, (3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 16, 16]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─ReLU: 2-2                         [-1, 16, 32, 32]          --
|    └─MaxPool2d: 2-3                    [-1, 16, 16, 16]          --
├─Sequential: 1-2                        [-1, 32, 8, 8]            --
|    └─Conv2d: 2-4                       [-1, 32, 16, 16]          4,640
|    └─ReLU: 2-5                         [-1, 32, 16, 16]          --
|    └─MaxPool2d: 2-6                    [-1, 32, 8, 8]            --
├─Sequential: 1-3                        [-1, 64, 4, 4]            --
|    └─Conv2d: 2-7                       [-1, 64, 8, 8]            18,496
|    └─ReLU: 2-8                         [-1, 64, 8, 8]            --
|    └─MaxPool2d: 2-9                    [-1, 64, 4, 4]            --
├─Flatten: 1-4                           [-1, 1024]                --
Total p

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        [-1, 16, 16, 16]          --
|    └─Conv2d: 2-1                       [-1, 16, 32, 32]          448
|    └─ReLU: 2-2                         [-1, 16, 32, 32]          --
|    └─MaxPool2d: 2-3                    [-1, 16, 16, 16]          --
├─Sequential: 1-2                        [-1, 32, 8, 8]            --
|    └─Conv2d: 2-4                       [-1, 32, 16, 16]          4,640
|    └─ReLU: 2-5                         [-1, 32, 16, 16]          --
|    └─MaxPool2d: 2-6                    [-1, 32, 8, 8]            --
├─Sequential: 1-3                        [-1, 64, 4, 4]            --
|    └─Conv2d: 2-7                       [-1, 64, 8, 8]            18,496
|    └─ReLU: 2-8                         [-1, 64, 8, 8]            --
|    └─MaxPool2d: 2-9                    [-1, 64, 4, 4]            --
├─Flatten: 1-4                           [-1, 1024]                --
Total p

**Train data**

In [11]:
# loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=learning_rate)

# train data
def train(train_loader, num_epochs):
  for epoch in range(num_epochs):
    for data, labels in train_loader:
      data = data.to(device)
      labels = lables.to(device)
      output = cnn_model(data)
      loss = criterion(output, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()